In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
root_url = 'https://cooljugator.com'
home_url = '/de/list/all'
r = requests.get(root_url + home_url)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
def create_verb_rec():
    pronouns = ['ich', 'du', 'er', 'wir', 'ihr', 'Sie']
    verb_rec = dict.fromkeys(['verb', 'translation', 'present', 'perfect', 'future'])
    for case in ['present', 'perfect', 'future']:
        verb_rec[case] = dict.fromkeys(pronouns)
    return verb_rec

In [4]:
pronouns = ['ich', 'du', 'er', 'wir', 'ihr', 'Sie']

verb_collection = []

for verb_link in soup.find_all('li', attrs={'class': 'item'}):
    verb_rec = create_verb_rec()
    verb, translation = verb_link.text.split(' - ')
    verb_rec['verb'] = verb
    verb_rec['translation'] = translation
    url_suffix = verb_link.find('a', href=True)['href']
    verb_url = root_url + url_suffix
    verb_soup = BeautifulSoup(requests.get(verb_url).text, 'html.parser')
    for case in ['present', 'perfect', 'future']:
        for i, pronoun in enumerate(pronouns):
            case_class = '{}{}'.format(case, i+1) 
            conjugated_verb = verb_soup.find('div', {"id": case_class}).find('div', {'class': 'meta-form'}).text
            verb_rec[case][pronoun] = conjugated_verb
    verb_collection.append(verb_rec)

In [5]:
len(verb_collection)

In [6]:
x = []
for vc in verb_collection:
    d = {}
    for k, v in vc.items():
        if isinstance(v, str):
            d[k] = v
        else:
            for ki, vi in v.items():
                d[(k, ki)] = vi
    x.append(d)

In [7]:
#using xlsx because it preserves utf8 better than csvs (and we have a lot of umlauts here!)
writer = pd.ExcelWriter('/Users/rob/Documents/verbs.xlsx', options={'encoding':'utf-8'})
pd.DataFrame(x).to_excel(writer, 'Sheet1')
writer.save()